# Principal Component Analysis
* `dim2` -- PCA Minka's MLE
* `dim3` -- PCA required Explained Variance

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# add path
import sys; import os; sys.path.append(os.path.realpath("../"))
import pandas as pd

In [3]:
# demo datasets
from datasets.demo1 import X_train, Y_train, fold_ids, X_valid, Y_valid, meta as meta_data
#meta_data

In [4]:
# transformer implementations
typ = 'dim2'

if typ is 'dim2':
    from verto.dim2 import trans, meta
elif typ is 'dim3':
    from verto.dim3 import trans, meta
    trans.set_params(**{'required_ev': 0.5})  # slowly increase from 0.1 towards 100%

In [5]:
meta

{'id': 'dim2',
 'name': 'PCA Minka MLE',
 'description': "Minka's MLE to guess n_components",
 'keywords': ['dimensionality reduction',
  'principal component anlysis',
  'StandardScaler',
  'PCA',
  'Minka MLE']}

In [6]:
trans  # __repr__

PcaMinka(prefix='minka')

In [7]:
print(trans)  # __str__

PcaMinka(prefix='minka')


## Transform

In [8]:
%%time
trans.fit(X_train)

CPU times: user 18.6 ms, sys: 2.03 ms, total: 20.7 ms
Wall time: 19.5 ms


PcaMinka(prefix='minka')

In [9]:
%%time
X_new = trans.transform(X_train)

CPU times: user 1.74 ms, sys: 1.49 ms, total: 3.24 ms
Wall time: 1.69 ms


In [10]:
df_new = pd.DataFrame(data=X_new, columns=trans.feature_names_)

## Evaluate
- check if the PCA components are "good" predictors
- eyeball the p-values of the logistic regression coefficients

In [11]:
df_new.head()

,minka_0,minka_1,minka_2,minka_3,minka_4,minka_5,minka_6,minka_7,minka_8,minka_9,...,minka_19,minka_20,minka_21,minka_22,minka_23,minka_24,minka_25,minka_26,minka_27,minka_28
0,-3.083984,-2.158203,-0.339844,-0.216675,1.544922,0.258057,0.571289,-0.137939,-0.100342,-0.269531,...,0.085938,-0.047119,0.070740,-0.029984,-0.083984,0.013786,-0.000401,-0.053009,0.030823,-0.013023
1,-2.183594,-0.617676,0.447266,-0.150879,2.859375,-0.353027,-0.098267,0.222168,-0.334961,-0.221436,...,0.087708,0.022522,-0.015366,-0.107239,-0.058014,0.070618,-0.030899,0.062744,0.045227,0.004559
2,2.050781,2.328125,1.168945,-1.814453,-0.115601,0.280762,0.211060,-0.176880,-0.740234,-0.214478,...,-0.048126,-0.083984,-0.183838,0.022339,0.027817,-0.056915,-0.176270,-0.029724,0.036804,0.004261
3,-2.808594,-0.950684,-0.305176,-0.380859,0.546875,1.203125,-0.277588,-0.093079,0.380371,0.372314,...,-0.007004,0.046143,-0.052399,-0.054749,0.038910,-0.038177,-0.007996,0.022980,0.006023,0.022491
4,-1.913086,-1.072266,0.498291,1.311523,-0.204224,0.064941,-0.051331,-0.019699,0.153931,-0.166260,...,-0.076294,-0.028442,0.053833,-0.118896,-0.095581,-0.017822,-0.066406,-0.013092,-0.007168,-0.005123


In [12]:
import statsmodels.api as sm
#lr = sm.Logit(Y_train, sm.add_constant(X_new)).fit()
lr = sm.Logit(Y_train, sm.add_constant(X_new)).fit_regularized(method='l1', alpha=.5)
print(lr.summary())

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.07083846334278182
            Iterations: 190
            Function evaluations: 190
            Gradient evaluations: 190
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  398
Model:                          Logit   Df Residuals:                      382
Method:                           MLE   Df Model:                           15
Date:                Sun, 31 Mar 2019   Pseudo R-squ.:                  0.9237
Time:                        13:23:29   Log-Likelihood:                -20.073
converged:                       True   LL-Null:                       -263.17
                                        LLR p-value:                 4.677e-94
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------

/Users/uh/cfs/venv-3.6.2/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/uh/cfs/venv-3.6.2/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/uh/cfs/venv-3.6.2/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
